# i-Eye Saffron Image Classifier

In [1]:
from keras.models import load_model
import ipywidgets as widgets
import io
from PIL import Image
import numpy as np
from keras.preprocessing.image import img_to_array
from IPython import display
import whatimage
import pyheif

In [2]:
model = load_model('vgg_model_4.h5')

2021-09-12 02:50:34.438005: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-12 02:50:34.439045: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-12 02:50:34.441435: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
def show_my_image():
    for name, file_info in btn_upload.value.items():
        img = decodeImage((file_info['content'])) 
    #return the image 
    return img.resize((400,400))

In [4]:
def decodeImage(bytesIo):
    fmt = whatimage.identify_image(bytesIo)
    if fmt in ['heic','heif']:
        i = pyheif.read_heif(bytesIo)
        pi = Image.frombytes(mode=i.mode, size=i.size, data=i.data)
    else:
        pi = Image.open(io.BytesIO(bytesIo))
    return pi

In [17]:
def predict(image: Image.Image):
    for name, file_info in btn_upload.value.items():
        img = decodeImage((file_info['content']))
    img = img_to_array(img.resize(size=(224,224)))
    img = np.expand_dims(img, axis = 0)
    img = img / 127.5 - 1.0    
    result = model.predict(img)    
    if result[0][0] <=0.50 :
        lbl_pred.value = "Predicted Class : " + "Non-Saffron"
    else:
        lbl_pred.value = "Predicted Class : " + "Saffron"
    saffron_prob.value = "probability of Saffron : " +str(result[0][0]*100) +"%"
    nsaffron_prob.value = "probability of Non-Saffron: " +str((1-result[0][0])*100) +"%"
    # clear the output
    out_pl.clear_output()
     # display the uploaded image
    with out_pl:
        display.display(show_my_image())

In [18]:
# Elements Of the App
# upload Button
btn_upload = widgets.FileUpload()
# output widget to display image 
out_pl = widgets.Output()
# label for the result of prediction
lbl_pred = widgets.Label()
saffron_prob = widgets.Label()
nsaffron_prob = widgets.Label()
# Button to classify the uploaded image 
btn_run = widgets.Button(description='Classify')
# event for classify button to predict the result and display uploaded image
btn_run.on_click(predict)

In [19]:
widgets.VBox([widgets.Label('Select Image'),btn_upload,btn_run,out_pl,lbl_pred,saffron_prob,nsaffron_prob],box_style='info')

VBox(box_style='info', children=(Label(value='Select Image'), FileUpload(value={}, description='Upload'), Butt…

0.8281099
0.19907242
